In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [ ]:
# TWES臺灣證券交易所_每日收盤行情(ETF)_HTML版
date = input("請輸入西元年月日，例：20240522")
# https://www.twse.com.tw/rwd/zh/afterTrading/MI_INDEX?date=20241009&type=0099P&response=html
url = "https://www.twse.com.tw/rwd/zh/afterTrading/MI_INDEX?date=" + date + "&type=0099P&response=html"
print(url)

In [ ]:
resp = requests.get(url)
resp

In [ ]:
soup = BeautifulSoup(resp.text, "lxml")
soup

In [ ]:
title = soup.find("thead").find("div").text
title

In [ ]:
len(title), type(title)

In [ ]:
title.partition(" ")[0]

In [ ]:
ths = [ th.text for th in soup.find("thead").find_all("th")[1:] ]
ths

In [ ]:
tds = [ [ td.text for td in tr.find_all("td") ] for tr in soup.find("tbody").find_all("tr") ]
tds

In [ ]:
datas = pd.DataFrame(tds, columns=ths)
datas

In [ ]:
datas.info()

In [144]:
# 將新增欄位，並指定欄位位置 → 漲跌價 = 漲跌(+/-) + 漲跌價差
datas.insert(loc=9, column="漲跌價", value=datas["漲跌(+/-)"]+datas["漲跌價差"])

In [ ]:
datas

In [146]:
# 刪除 漲跌(+/-) 、 漲跌價差 這 2 個欄位。iloc的欄位置→10,11；inplace=True→改變原始資料(直接覆寫)
datas.drop(datas.iloc[:,[10, 11, 16]],axis=1,inplace=True)

In [ ]:
datas

In [148]:
datas["成交股數"] = datas["成交股數"].apply(lambda x:eval(x.replace(",","")))

In [ ]:
datas.info()

In [ ]:
datas.sort_values("成交股數", ascending=False)[:20]

In [156]:
datas.drop(datas.iloc[:, 10:14], axis=1, inplace=True)

In [ ]:
columns = datas.columns
columns

In [ ]:
columns[:10]

In [ ]:
datas.values